In [19]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score
import numpy as np
from utils.training import train
from data_loaders.imagenet import Train, Val
from models.resnet_real import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
from models.resnet_quat import ResNet18_quat, ResNet34_quat, ResNet50_quat, ResNet101_quat, ResNet152_quat

In [60]:
# import numpy as np
# from data_loaders.ILSVRC import Train, Val
# import torch
# import torch.nn.functional as F
# from tqdm import tqdm, trange
# from utils.training import train
# from utils.accuracy import all_acc_prunemodels, top5acc_check, all_acc_model
# hparams = {
#     "batch_size": 256,
#     "num_epochs": 15,
#     "model": "ResNet18",
#     "dataset": "ILSVRC",
#     "optimizer": "sgd",
#     "learning_rate": 0.1,
#     "gpu": 0,
#     "pruning_percentages": [100, 80, 64, 50, 32, 25, 20, 16, 13, 10, 8, 6, 4],
#     "batches" : 1
# }

# model_name = "RN18_real"
# models_folder = f"/home/aritra/project/quatLT23/6_pruning/saved_models/{model_name}"
# save_name = f"acc_{model_name}_prunes_trial2.txt"
# log = False #turn this on!

# model_path = "/home/aritra/project/quatLT23/6_pruning/saved_models/untitled/4-ResNet152_imagenet64_B=256_O=sgd_ll=0.1_E=35.pth"

# if log:
#     import wandb
#     wandb.init(project="QuatLT23", name="LAST TRIAL DELETE", config=hparams)
# # ILSVRC_RN18_realprunes_trial2 accuracies
# training_generator = torch.utils.data.DataLoader(Train(50000), shuffle=True, batch_size=hparams["batch_size"], num_workers=4)
# validation_generator = torch.utils.data.DataLoader(Val(),      shuffle=True, batch_size=hparams["batch_size"], num_workers=4)

# acc_dict = all_acc_model(
#     model_path,
#     training_generator,
#     validation_generator,
#     GPU = torch.device(f'cuda:0'),
#     # save = save_name,
#     batches=10
# )

# print(acc_dict)


# '''
# 4:  (43.7890625, 62.9296875, 45.4296875, 70.0390625)
# 10: (43.9453125, 63.0859375, 45.5078125, 70.9765625)
# 20: (44.1015625, 63.671875, 46.5234375, 71.875)
# 50: (43.0078125, 61.40625, 44.9609375, 70.1953125)
# 80: (43.203125, 63.3984375, 45.5078125, 70.9375)
# 100:(43.828125, 63.359375, 44.4921875, 69.0234375)
# '''




Train:  10%|█         | 1/10 [00:04<00:40,  4.49s/batch]


OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 47.54 GiB total capacity; 45.84 GiB already allocated; 114.00 MiB free; 46.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
filename = "/home/aritra/project/quatLT23/6_pruning/acc_RN18_real_prunes_trial2.txt"  # Replace with the name of your file
my_dict = {}

with open(filename, "r") as f:
    i = 0
    for line in f:
        if i == 0: 
            i += 1
            continue
        items = line.strip().split(" ")
        key = items[0]
        values = [float(val) for val in items[1:]]
        my_dict[key] = values

print(my_dict)

{'/4/1.pth': [10.546875, 26.953125, 19.921875, 39.0625], '/4/2.pth': [19.921875, 34.765625, 31.25, 57.8125], '/4/3.pth': [27.734375, 43.359375, 37.109375, 64.0625], '/4/4.pth': [28.90625, 45.3125, 39.0625, 63.671875], '/4/5.pth': [32.8125, 54.6875, 41.40625, 67.96875], '/4/6.pth': [32.8125, 52.34375, 44.921875, 67.578125], '/4/7.pth': [35.546875, 54.296875, 44.140625, 67.578125], '/4/8.pth': [40.625, 58.59375, 49.21875, 68.359375], '/4/9.pth': [38.28125, 57.8125, 48.828125, 68.75], '/4/10.pth': [39.84375, 60.15625, 46.09375, 69.53125], '/4/11.pth': [37.890625, 58.984375, 48.828125, 70.703125], '/4/12.pth': [46.875, 61.328125, 46.09375, 69.53125], '/4/13.pth': [42.1875, 62.5, 47.65625, 71.875], '/4/14.pth': [42.1875, 61.328125, 44.140625, 69.140625], '/4/15.pth': [41.40625, 60.9375, 44.140625, 68.75], '/6/1.pth': [10.546875, 26.953125, 19.921875, 39.0625], '/6/2.pth': [19.921875, 34.765625, 31.25, 57.8125], '/6/3.pth': [27.734375, 43.359375, 37.109375, 64.0625], '/6/4.pth': [28.90625, 4

In [38]:
train_acc = {}
test_acc = {}
train_acc_top5 = {}
test_acc_top5 = {}
prune_ratios = []
for key in my_dict:
    prune_ratios.append(int(key.split("/")[-2])) if int(key.split("/")[-2]) not in prune_ratios else prune_ratios

print(prune_ratios)

for key in my_dict:
    prune_ratio = int(key.split("/")[-2])
    if prune_ratio not in train_acc:
        train_acc[prune_ratio] = [my_dict[key][0]]
        test_acc[prune_ratio] = [my_dict[key][2]]
        train_acc_top5[prune_ratio] = [my_dict[key][1]]
        test_acc_top5[prune_ratio] = [my_dict[key][3]]
    else:
        train_acc[prune_ratio].append(my_dict[key][0])
        test_acc[prune_ratio].append(my_dict[key][2])
        train_acc_top5[prune_ratio].append(my_dict[key][1])
        test_acc_top5[prune_ratio].append(my_dict[key][3])

# train_acc = [train_acc[key] for key in train_acc]
# test_acc = [test_acc[key] for key in test_acc]
# train_acc_top5 = [train_acc_top5[key] for key in train_acc_top5]
# test_acc_top5 = [test_acc_top5[key] for key in test_acc_top5]
        
print(train_acc)
print(test_acc)
print(train_acc_top5)
print(test_acc_top5)



[4, 6, 8, 10, 13, 16, 20, 25, 32, 50, 64, 80, 100]
{4: [10.546875, 19.921875, 27.734375, 28.90625, 32.8125, 32.8125, 35.546875, 40.625, 38.28125, 39.84375, 37.890625, 46.875, 42.1875, 42.1875, 41.40625], 6: [10.546875, 19.921875, 27.734375, 28.90625, 32.8125, 32.8125, 35.546875, 40.625, 38.28125, 39.84375, 37.890625, 46.875, 42.1875, 42.1875, 41.40625], 8: [10.546875, 19.921875, 27.734375, 28.90625, 32.8125, 32.8125, 35.546875, 40.625, 38.28125, 39.84375, 37.890625, 46.875, 42.1875, 42.1875, 41.40625], 10: [10.546875, 19.921875, 27.734375, 28.90625, 32.8125, 32.8125, 35.546875, 40.625, 38.28125, 39.84375, 37.890625, 46.875, 42.1875, 42.1875, 41.40625], 13: [10.546875, 19.921875, 27.734375, 28.90625, 32.8125, 32.8125, 35.546875, 40.625, 38.28125, 39.84375, 37.890625, 46.875, 42.1875, 42.1875, 41.40625], 16: [10.546875, 19.921875, 27.734375, 28.90625, 32.8125, 32.8125, 35.546875, 40.625, 38.28125, 39.84375, 37.890625, 46.875, 42.1875, 42.1875, 41.40625], 20: [10.546875, 19.921875, 27.734

In [22]:
# individual train_acc plots
for prune_ratio in prune_ratios:
    plt.plot(train_acc[prune_ratio], label=f"train_acc_{prune_ratio}%")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy(%)")
    plt.title(f"Train Accuracy vs Epochs for {prune_ratio}%")
    plt.legend()
    plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/train_acc_{prune_ratio}.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [23]:
# individual test_acc plots
for prune_ratio in prune_ratios:
    plt.plot(test_acc[prune_ratio], label=f"test_acc_{prune_ratio}%")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy(%)")
    plt.title(f"Test Accuracy vs Epochs for {prune_ratio}%")
    plt.legend()
    plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/test_acc_{prune_ratio}.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [24]:
# individual train_acc_top5 plots
for prune_ratio in prune_ratios:
    plt.plot(train_acc_top5[prune_ratio], label=f"train_acc_top5_{prune_ratio}%")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy(%)")
    plt.title(f"Train Accuracy Top5 vs Epochs for {prune_ratio}%")
    plt.legend()
    plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/train_acc_top5_{prune_ratio}.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [25]:
# individual test_acc_top5 plots
for prune_ratio in prune_ratios:
    plt.plot(test_acc_top5[prune_ratio], label=f"test_acc_top5_{prune_ratio}%")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy(%)")
    plt.title(f"Test Accuracy Top5 vs Epochs for {prune_ratio}%")
    plt.legend()
    plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/test_acc_top5_{prune_ratio}.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [47]:
# combined train_acc plots
for prune_ratio in prune_ratios:
    plt.plot(train_acc[prune_ratio], label=f"{prune_ratio}% w_left")
plt.xlabel("Epochs")
plt.ylabel("Accuracy(%)")
plt.title(f"Train Accuracy vs Epochs")
plt.legend()
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/train_acc_combined.png")
plt.clf()

<Figure size 640x480 with 0 Axes>

In [48]:
# combined test_acc plots
for prune_ratio in prune_ratios:
    plt.plot(test_acc[prune_ratio], label=f"{prune_ratio}% w_left")
plt.xlabel("Epochs")
plt.ylabel("Accuracy(%)")
plt.title(f"Test Accuracy vs Epochs")
plt.legend()
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/test_acc_combined.png")
plt.clf()


<Figure size 640x480 with 0 Axes>

In [49]:
# combined train_acc_top5 plots
for prune_ratio in prune_ratios:
    plt.plot(train_acc_top5[prune_ratio], label=f"{prune_ratio}% w_left")
plt.xlabel("Epochs")
plt.ylabel("Accuracy(%)")
plt.title(f"Train Accuracy Top5 vs Epochs")
plt.legend()
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/train_acc_top5_combined.png")
plt.clf()


<Figure size 640x480 with 0 Axes>

In [50]:
# combined test_acc_top5 plots
for prune_ratio in prune_ratios:
    plt.plot(test_acc_top5[prune_ratio], label=f"{prune_ratio}% w_left")
plt.xlabel("Epochs")
plt.ylabel("Accuracy(%)")
plt.title(f"Test Accuracy Top5 vs Epochs")
plt.legend()
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/test_acc_top5_combined.png")
plt.clf()

<Figure size 640x480 with 0 Axes>

In [43]:
# final train_acc plot for prunes
final_train_acc = [train_acc[prune_ratio][-1] for prune_ratio in prune_ratios]
plt.plot(prune_ratios, final_train_acc)
plt.xlabel("Prune Ratio(%)")
plt.ylabel("Accuracy(%)")
plt.title(f"Final Train Accuracy vs Prune Ratio")
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/final_train_acc.png")
plt.clf()
    

<Figure size 640x480 with 0 Axes>

In [44]:
# final test_acc plot for prunes
final_test_acc = [test_acc[prune_ratio][-1] for prune_ratio in prune_ratios]
plt.plot(prune_ratios, final_test_acc)
plt.xlabel("Prune Ratio(%)")
plt.ylabel("Accuracy(%)")
plt.title(f"Final Test Accuracy vs Prune Ratio")
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/final_test_acc.png")
plt.clf()

<Figure size 640x480 with 0 Axes>

In [45]:
# final train_acc_top5 plot for prunes
final_train_acc_top5 = [train_acc_top5[prune_ratio][-1] for prune_ratio in prune_ratios]
plt.plot(prune_ratios, final_train_acc_top5)
plt.xlabel("Prune Ratio(%)")
plt.ylabel("Accuracy(%)")
plt.title(f"Final Train Accuracy Top5 vs Prune Ratio")
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/final_train_acc_top5.png")
plt.clf()


<Figure size 640x480 with 0 Axes>

In [46]:
# final test_acc_top5 plot for prunes
final_test_acc_top5 = [test_acc_top5[prune_ratio][-1] for prune_ratio in prune_ratios]
plt.plot(prune_ratios, final_test_acc_top5)
plt.xlabel("Prune Ratio(%)")
plt.ylabel("Accuracy(%)")
plt.title(f"Final Test Accuracy Top5 vs Prune Ratio")
plt.savefig(f"/home/aritra/project/quatLT23/6_pruning/saved_figs/RN_158_real_prunes/final_test_acc_top5.png")
plt.clf()

<Figure size 640x480 with 0 Axes>